In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
print("API Key configured:", "OPENAI_API_KEY" in os.environ)
from openai import OpenAI
client = OpenAI()

API Key configured: True


In [2]:
response = client.chat.completions.create(
  model='gpt-4o-mini', #omni model
  messages=[
    {'role':'user', 'content': "파이썬에 대해 알려줘"}
  ]
)

In [3]:
response.choices[0].message.content

'파이썬(Python)은 높은 수준의 프로그래밍 언어로, 읽기 쉽고 작성하기 간편한 문법을 가지고 있습니다. 1991년 귀도 반 로썸(Guido van Rossum)에 의해 개발되었으며, 이후 지속적으로 발전해왔습니다. 파이썬의 주요 특징은 다음과 같습니다:\n\n1. **가독성**: 파이썬의 코드 구조는 명확하고 간결하여 쉽게 이해할 수 있습니다. 이는 다른 언어에 비해 생산성을 높이는 데 기여합니다.\n\n2. **다양한 용도**: 웹 개발, 데이터 분석, 인공지능, 과학 계산, 자동화 스크립트 등 다양한 분야에서 사용됩니다.\n\n3. **대규모 라이브러리**: 파이썬은 많은 외부 라이브러리와 프레임워크를 지원합니다. 예를 들어, NumPy와 Pandas는 데이터 분석에, Flask와 Django는 웹 개발에 유용합니다.\n\n4. **크로스 플랫폼**: Windows, macOS, Linux 등 다양한 운영 체제에서 실행할 수 있습니다.\n\n5. **커뮤니티 지원**: 풍부한 문서와 활발한 사용자 커뮤니티가 있어 문제 해결 및 학습에 도움이 됩니다.\n\n6. **동적 타이핑**: 변수를 선언할 때 타입을 명시하지 않아도 되며, 실행 도중 타입이 결정됩니다.\n\n파이썬의 간단한 예시는 다음과 같습니다:\n\n```python\n# Hello, World! 출력하기\nprint("Hello, World!")\n\n# 리스트와 반복문 사용 예\nnumbers = [1, 2, 3, 4, 5]\nfor number in numbers:\n    print(number)\n```\n\n파이썬을 배우기 위해서는 기본 문법을 익히고, 다양한 프로젝트를 통해 실습하는 것이 좋습니다. 이 외에도 궁금한 점이 있으면 질문해 주세요!'

In [4]:
message = {"role": "user", "content": "안녕하세요, 오늘 날씨는 어떤가요?"}
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ message ]
)
assistant_reply = response.choices[0].message.content
print("ChatGPT 답변:", assistant_reply)

ChatGPT 답변: 안녕하세요! 제가 실시간 정보를 제공할 수는 없지만, 오늘의 날씨를 확인하시려면 날씨 앱이나 웹사이트를 이용해보시는 것이 좋습니다. 지역에 따라 날씨가 다르니, 구체적인 지역을 검색하시면 정확한 정보를 얻을 수 있습니다. 도움이 필요하시면 말씀해 주세요!


# 출력 형식 지정

In [5]:
message = {"role": "user", "content": "오늘 서울 날씨를 json 형태로 제공해. 키는 weather, temperature 로 설정해 "}
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ message ],
    response_format={'type':'json_object'}
)
assistant_reply = response.choices[0].message.content
print("ChatGPT 답변:", assistant_reply)

ChatGPT 답변: {
  "weather": "맑음",
  "temperature": "20도"
}


In [6]:
import json
data = json.loads(assistant_reply)
data.keys()

dict_keys(['weather', 'temperature'])

In [ ]:
# 도시명... 의 날씨는 현재 .... 이고, 온도는 ... 입니다.

# CoT 프롬프트 

In [12]:
question = "철수는 사과 10개를 가지고 있었는데, 그 중 3개를 먹은 후 5개를 더 구입했습니다. 철수가 현재 가지고 있는 사과는 몇 개일까요?"
response = client.chat.completions.create(
  messages=[{'role':'user', 'content': question}],
  model='gpt-4o-mini',
  temperature=0
)
print('CoT 적용전 : ',response.choices[0].message.content )

CoT 적용전 :  철수는 처음에 사과 10개를 가지고 있었습니다. 그 중 3개를 먹었으므로 남은 사과는 10 - 3 = 7개입니다. 이후 5개를 더 구입했으므로 현재 가지고 있는 사과는 7 + 5 = 12개입니다. 

따라서 철수가 현재 가지고 있는 사과는 12개입니다.


In [13]:
cot_prompt = '(생각을 단계별로 진행해)\n'+ question
response = client.chat.completions.create(
  messages=[{'role':'user', 'content': cot_prompt}],
  model='gpt-4o-mini',
  temperature=0
)
print('CoT 적용후 : ',response.choices[0].message.content )

CoT 적용후 :  단계별로 생각해보겠습니다.

1. 철수는 처음에 사과 10개를 가지고 있었습니다.
2. 그 중 3개를 먹었습니다. 
   - 10개 - 3개 = 7개
3. 이후에 5개를 더 구입했습니다.
   - 7개 + 5개 = 12개

따라서, 철수가 현재 가지고 있는 사과는 12개입니다.


# Self consistency 기법

In [14]:
question = "어떤 수를 3배한 결과에 4를 더하면 19가 됩니다. 그 수는 무엇일까요? 단계별로 풀어보세요."
message = [{'role':'user', 'content':question}]
answers = []

for i in range(5):
  response = client.chat.completions.create(
    messages=message,
    model='gpt-4o-mini',
    temperature=1.0
  ) 
  answer = response.choices[0].message.content.strip()
  answers.append(answer)
  print(f'응답 {i+1} : {answer}')

응답 1 : 주어진 문제를 수식으로 표현해 보겠습니다. 우리가 찾고자 하는 수를 \( x \)라고 하겠습니다. 문제에서 주어진 내용을 수식으로 나타내면 아래와 같습니다:

\[
3x + 4 = 19
\]

이제 이 수식을 단계별로 풀어보겠습니다.

1. **4를 왼쪽으로 이동**: 먼저 \( 3x \)를 구하기 위해 양쪽에서 4를 빼줍니다.

\[
3x + 4 - 4 = 19 - 4
\]
\[
3x = 15
\]

2. **양쪽을 3으로 나누기**: 이제 \( x \)를 구하기 위해 양쪽을 3으로 나눕니다.

\[
\frac{3x}{3} = \frac{15}{3}
\]
\[
x = 5
\]

결과적으로, 어떤 수는 \( 5 \)입니다. 

**검증**: 이 값을 원래 방정식에 대입하여 제대로 계산되었는지 확인해 보겠습니다.

\[
3 \times 5 + 4 = 15 + 4 = 19
\]

따라서, 원래 방정식이 성립하므로, 그 수는 \( \boxed{5} \)가 맞습니다.
응답 2 : 문제를 단계별로 풀어봅시다.

1. **문제 이해하기**: 어떤 수를 \( x \)라고 할 때, 그 수를 3배한 결과에 4를 더하면 19가 된다고 했습니다. 이를 수학적으로 표현하면 다음과 같습니다.
   \[
   3x + 4 = 19
   \]

2. **방정식 설정하기**: 위에서 만든 방정식은 다음과 같습니다.
   \[
   3x + 4 = 19
   \]

3. **방정식 풀기**: 먼저 양변에서 4를 빼겠습니다.
   \[
   3x + 4 - 4 = 19 - 4
   \]
   이 계산을 하면,
   \[
   3x = 15
   \]

4. **\( x \) 구하기**: 이제 양변을 3으로 나눕니다.
   \[
   x = \frac{15}{3}
   \]
   계산하면,
   \[
   x = 5
   \]

5. **답 확인하기**: 구한 값을 원래의 조건에 대입하여 확인해보겠습니다.
   \[
   3 \times 5 + 4 = 15 + 4 = 19
   \

In [17]:
final_answer = [ans.split()[-1] for ans in answers]
import collections
counter = collections.Counter(final_answer)
final_ans = counter.most_common(1)[0][0]
print(f'최종 답 : {final_ans}')

최종 답 : **5**입니다.


# Reflexion (자기 피드백 반복 프롬프트 설계)

In [19]:
question = "7의 2승에 5를 곱한 값은 무엇인가?"
response1 = client.chat.completions.create(
    messages=[
        {"role": "user", "content": question}
    ],
    model="gpt-4o-mini",
    temperature=0
)
answer = response1.choices[0].message.content.strip()  #모델이 출력한 응답

correct_answer =  245 #우리가 주는 피드백
if str(correct_answer) in answer :  #text
  feedback = '정답입니다.'
  need_retry = False
else : 
  feedback = '틀렸어요. 힌트는 7의 제곱값을 정확히 구한 다음 곱하세요'
  need_retry = True

print(feedback)

정답입니다.


# 틀린 답을 출력하는 모델은?

In [20]:
# 2차 답변 생성
if need_retry :
  retry_prompt = f'이전 답변 : {answer} , 피드백: {feedback} \n 다시 답을 구하세요'
  response2 = client.chat.completions.create(
    messages=[
        {"role": "user", "content": retry_prompt }
    ],
    model="gpt-4o-mini",
    temperature=0
  )
  answer2 = response2.choices[0].message.content.strip()  #모델이 출력한 응답
  print(answer2)

In [ ]:
prompt = "기후 변화의 원인과 해결 방안에 대해 간략한 에세이를 작성하시오."

In [ ]:
# 에세이 작성 1차 시도
prompt = "기후 변화의 원인과 해결 방안에 대해 간략한 에세이를 작성하시오."
response_essay1 = client.chat.completions.create(
    messages=[
        {"role": "user", "content": prompt}
    ],
    model="gpt-4o-mini",
    temperature=0
)
essay1 = response_essay1.choices[0].message.content
print("1차 에세이:\n", essay1)

In [ ]:
# 피드백 작성 (여기서는 사람이 수동으로 평가하여 문자열 작성한다고 가정)
feedback = (
    "피드백:\n"
    " - 서론에서 주제 소개가 부족합니다.\n"
    " - 원인에 대한 설명이 모호하며 구체적 예시와 근거를 제시할 통계자료가 없습니다.\n"
    " - 해결 방안을 최소 10가지 이상 제시해주세요.\n"
    "위 사항을 반영하여 에세이를 수정해주세요."
)
# 개선 프롬프트 구성
retry_prompt = essay1 + "\n\n" + feedback
response_essay2 = client.chat.completions.create(
    messages=[
        {"role": "user", "content": retry_prompt}
    ],
    model="gpt-4o-mini",
    temperature=0
)
essay2 = response_essay2.choices[0].message.content
print("2차 에세이:\n", essay2)

In [ ]:
# 피드백 작성 (여기서는 사람이 수동으로 평가하여 문자열 작성한다고 가정)
feedback_2 = (
    "피드백:\n"
    " - 관련 사실을 실증할만한 근거의 통계 자료가 없습니다.\n"
    "위 사항을 반영하여 에세이를 수정해주세요."
)
# 개선 프롬프트 구성
retry_prompt_2 = essay1 + "\n\n" + feedback_2
response_essay3 = client.chat.completions.create(
    messages=[
        {"role": "user", "content": retry_prompt_2}
    ],
    model="gpt-4o-mini",
    temperature=0
)
essay3 = response_essay3.choices[0].message.content
print("3차 에세이:\n", essay3)